In [2]:
import pandas as pd
from skimage.color import rgb2hed
import skimage
import numpy as np
from skimage.io import imread
from skimage.color import rgba2rgb
from skimage.transform import resize

In [4]:
data=pd.read_csv('generated_dataframe.csv')
data.head()

,Episode,Trial_idx_epi,State,Stim,Correct_response,Trial_id,path
0,e1,1,reversed,face,1,1,/Users/zhengyuanrui/Sim_reversal/2_Data/IMG/tr...
1,e1,2,stable,face,1,2,/Users/zhengyuanrui/Sim_reversal/2_Data/IMG/tr...
2,e1,3,stable,face,1,3,/Users/zhengyuanrui/Sim_reversal/2_Data/IMG/tr...
3,e1,4,stable,face,1,4,/Users/zhengyuanrui/Sim_reversal/2_Data/IMG/tr...
4,e1,5,stable,face,1,5,/Users/zhengyuanrui/Sim_reversal/2_Data/IMG/tr...


In [5]:
#假设我们用path trial_id去预测 Correct_response	

In [6]:
def load_image(image_path,sizes=(28,28)):

    image=imread(image_path)
    if image.shape[-1]==4:
        I = rgba2rgb(image) #read sample RGB image
    else:
        I=image/255.
    I_resized = resize(I,sizes, anti_aliasing=True)
    return I_resized

In [7]:
#图片列表
images=np.stack([load_image(path) for path in data['path']])

trail_id=data['Trial_id'].values

Correct_response=data['Correct_response'].values

In [8]:
images.shape,trail_id.shape,Correct_response.shape

((24, 28, 28, 3), (24,), (24,))

In [10]:
import tensorflow as tf

2024-03-28 19:58:24.986105: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
image_input=tf.keras.layers.Input(shape=(28,28,3),name='image')
image_out=tf.keras.layers.Conv2D(16,kernel_size=3)(image_input)
image_out=tf.keras.layers.Flatten()(image_out)
image_out=tf.keras.layers.Dense(1)(image_out)

series_input=tf.keras.layers.Input(shape=(1,),name='series')
series_out=tf.keras.layers.Dense(1)(series_input)

outputs=tf.keras.layers.Concatenate(axis=-1)([image_out,series_out])

tf_model=tf.keras.Model(inputs=[image_input,series_input],outputs=outputs)

In [23]:
tf_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
loss=tf.keras.losses.SparseCategoricalCrossentropy(),
metrics=['acc'])
tf_model.fit(
    {"image": images, "series": trail_id},
     Correct_response,
    epochs=2,
    batch_size=32,
)

Epoch 1/2
1/1 [==============================] - 1s 577ms/step - loss: 11.4170 - acc: 0.2917
Epoch 2/2
1/1 [==============================] - 0s 6ms/step - loss: 11.4170 - acc: 0.2917


# torch

In [24]:
import torch
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms,models

class CustomImageDataset(Dataset):
    #数据加载器
    def __init__(self, images,ids,labels):

        self.images= images
        self.ids=ids
        self.labels = labels


    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
            
        I=self.images[idx]
        ID=self.ids[idx]
        label = self.labels[idx]
        
        I = torch.tensor(I,dtype=torch.float).permute(2, 0, 1)
        ID = torch.tensor(ID, dtype=torch.float)
        label = torch.tensor(label, dtype=torch.float)
        
        return I,ID, label


In [25]:
dataset = CustomImageDataset(images,trail_id, Correct_response)
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [26]:
for img,ID,label in train_dataloader:
    print(img.shape,ID.shape,label.shape)
    break

torch.Size([24, 3, 28, 28]) torch.Size([24]) torch.Size([24])
